In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import folium
from folium.plugins import HeatMap

In [2]:
pd.set_option('display.max_rows', None)


In [4]:
import json

# Abra o arquivo JSON e leia seu conteúdo com a codificação 'utf-8-sig'
with open('G:/felip/Documents/Git/analyser/data/municipios.json', 'r', encoding='utf-8-sig') as arquivo_json:
    dados = json.load(arquivo_json)
df = pd.DataFrame(dados)
# 'dados' agora contém o conteúdo do JSON como um dicionário em Python
df.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [5]:
# Abra o arquivo JSON e leia seu conteúdo com a codificação 'utf-8-sig'
with open('G:/felip/Documents/Git/analyser/data/estados.json', 'r', encoding='utf-8-sig') as arquivo_json:
    dados = json.load(arquivo_json)
est = pd.DataFrame(dados)
# 'dados' agora contém o conteúdo do JSON como um dicionário em Python
est.rename(columns={'nome':'estado'}, inplace=True)


In [6]:
df = df.merge(est[['uf', 'estado', 'codigo_uf']], on= 'codigo_uf')

In [7]:
import unicodedata

# Suponha que você já tenha um DataFrame chamado df

# Função para remover acentos
def remover_acentos(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

# Função para processar a coluna "nome"
def processar_coluna_nome(nome):
    # Remove acentos
    nome_sem_acentos = remover_acentos(nome)
    # Converte para maiúsculas
    nome_maiusculo = nome_sem_acentos.upper()
    # Substitui apóstrofos por espaços
    nome_processado = nome_maiusculo.replace("'", " ")
    return nome_processado

# Aplica a função de processamento à coluna "nome"
df['nome'] = df['nome'].apply(processar_coluna_nome)
df['estado'] = df['estado'].apply(processar_coluna_nome)

df.rename(columns={'nome': 'cidade'}, inplace=True)

In [8]:
df.to_csv('data/cidades.csv', sep= ';', index= False)

In [27]:
base_vendedores = pd.read_csv('data/vendedores.csv', sep=';')
base_pracas = pd.read_csv('data/pracas.csv', sep=';')
base_desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
base_pedidos = pd.read_csv('data/pedidos.csv', sep=';')
base_fechamentos = pd.read_csv('data/fechamentos.csv', sep=';')
base_sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
base_artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
base_gruposArtigos = pd.read_csv('data/grupos.csv', sep=';')
base_artigos = pd.read_csv('data/artigos.csv', sep=';')
base_artigosPedido['ano_mes_cobranca'] = pd.to_datetime(base_artigosPedido['ano_mes_cobranca'], format='%m/%Y').dt.date
base_desempenhoPraca['ano_mes_cobranca'] = pd.to_datetime(base_desempenhoPraca['ano_mes_cobranca'], format='%Y-%m').dt.date
base_pedidos['ano_mes_cobranca'] = pd.to_datetime(base_pedidos['ano_mes_cobranca'], format='%m/%Y').dt.date
geoCidades = pd.read_csv('data/cidades.csv', sep=';')

In [28]:
base_pedidos = base_pedidos.merge(base_sacoleiras[['id_sacoleira', 'cidade', 'uf']], on= 'id_sacoleira')
base_pedidos = base_pedidos.merge(geoCidades[['uf','cidade', 'latitude','longitude']], on=['cidade', 'uf'])

In [29]:
pedidos_ativos = base_pedidos[base_pedidos['status'] == 'DIGITADO']
vendedoras_ativas = pedidos_ativos['id_sacoleira'].unique() # type: ignore

In [30]:
pedidos_ativos_X_cidade = pedidos_ativos.groupby('cidade').agg({'id_sacoleira': 'count','valor_total_pedido': 'sum'})
pedidos_ativos_X_cidade = pedidos_ativos_X_cidade[pedidos_ativos_X_cidade['valor_total_pedido'] > 0]
pedidos_ativos_X_cidade['VMS'] = pedidos_ativos_X_cidade['valor_total_pedido']/pedidos_ativos_X_cidade['id_sacoleira']
pedidos_ativos_X_cidade['% Distribuição'] = (pedidos_ativos_X_cidade['valor_total_pedido'] / pedidos_ativos_X_cidade['valor_total_pedido'].sum()) * 100

In [31]:
#pedidos_ativos_X_cidade.sort_values('valor_total_pedido', ascending= False)

In [32]:
cadastros_ativos = base_sacoleiras.query('id_sacoleira in @vendedoras_ativas')

In [33]:
import folium
from folium.plugins import HeatMap

# Suponha que você tenha um DataFrame chamado 'pedidos_ativos' com as colunas 'latitude', 'longitude' e 'faturamento'

# Ajuste a escala dos valores de faturamento para cobrir uma faixa maior
min_faturamento = pedidos_ativos['valor_total_pedido'].min()
max_faturamento = pedidos_ativos['valor_total_pedido'].max()
pedidos_ativos['valor_total_pedido_scaled'] = (pedidos_ativos['valor_total_pedido'] - min_faturamento) / (max_faturamento - min_faturamento)

# Crie um objeto de mapa
m = folium.Map(location=[pedidos_ativos['latitude'].mean(), pedidos_ativos['longitude'].mean()], zoom_start=6)

# Crie uma lista de listas contendo latitudes, longitudes e faturamentos escalados
heat_data = [[row['latitude'], row['longitude'], row['valor_total_pedido_scaled']] for index, row in pedidos_ativos.iterrows()]

# Crie o mapa de calor
HeatMap(heat_data).add_to(m)

# Salve o mapa em um arquivo HTML
m.save('mapa_de_calor.html')

# Exiba o mapa no Jupyter Notebook (opcional)
m

C:\Users\felip\AppData\Local\Temp\ipykernel_11892\1765228090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos_ativos['valor_total_pedido_scaled'] = (pedidos_ativos['valor_total_pedido'] - min_faturamento) / (max_faturamento - min_faturamento)


In [34]:
periodo_i = None
periodo_f = None
pedidos_periodo = base_pedidos.query("")

ValueError: expr cannot be an empty string

In [ ]:
# Suponha que você tenha um DataFrame chamado 'pedidos_ativos' com as colunas 'latitude', 'longitude' e 'faturamento'

# Ajuste a escala dos valores de faturamento para cobrir uma faixa maior
min_faturamento = pedidos_ativos['valor_total_pedido'].min()
max_faturamento = pedidos_ativos['valor_total_pedido'].max()
pedidos_ativos['valor_total_pedido_scaled'] = (pedidos_ativos['valor_total_pedido'] - min_faturamento) / (max_faturamento - min_faturamento)

# Crie um objeto de mapa
m = folium.Map(location=[pedidos_ativos['latitude'].mean(), pedidos_ativos['longitude'].mean()], zoom_start=6)

# Crie uma lista de listas contendo latitudes, longitudes e faturamentos escalados
heat_data = [[row['latitude'], row['longitude'], row['valor_total_pedido_scaled']] for index, row in pedidos_ativos.iterrows()]

# Crie o mapa de calor
HeatMap(heat_data).add_to(m)

# Salve o mapa em um arquivo HTML
m.save('mapa_de_calor.html')

# Exiba o mapa no Jupyter Notebook (opcional)
m